In [10]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    max_tokens=1000,
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

examples = [
    {
        "question": "집에서 코딩을 할 때 가지고 있으면 좋은 아이템을 알려줘.",
        "answer": """
            집에서 코딩을 할 때에는 다음과 같은 아이템을 추천합니다:
            -키보드 손목 받침대
            -좋은 의자 (패브릭 소재 편안한 의자)
            -좋은 키보드 (기계식 무소음 키보드)
            -좋은 마우스 (버티컬 마우스)
            -좋은 듀얼 모니터
            -좋은 컴퓨터 (1660 super 이상, i5 12600kf 이상)
        """
    },
    {
        "question": "카페에서 코딩을 할 때 가지고 있으면 좋은 아이템을 알려줘.",
        "answer": """
            카페에서 코딩을 할 때에는 다음과 같은 아이템을 추천합니다:
            -에어팟 (소음 최소화)
            -맥북 (에어팟과 호환 가능, 코딩 할 때 쓰기 좋은 노트북)
            -무선 키보드 (가볍고 편한 무소음 키보드)
            -무선 마우스 (가볍고 조용한 무소음 축 마우스)
            -마우스 패드 (작고 휴대성이 좋은 마우스 패드)
        """
    },
    {
        "question": "수영장에 갈 때 가지고 가면 좋은 아이템을 알려줘.",
        "answer": """
            수영장에 갈 때에는 다음과 같은 아이템을 추천합니다:
            -수경 (수영을 위한 아이템)
            -수영모 (머리카락 빠짐 방지)
            -수영복 (수영을 위한 아이템)
        """
    },
]

exmaple_prompt = PromptTemplate.from_template("Human: {question}\nAI: {answer}")

prompt = FewShotPromptTemplate(
    example_prompt=exmaple_prompt,
    examples=examples,
    suffix="Human: {purpose} 가지고 가면 좋은 아이템을 알려줘.",
    input_variables=["purpose"]

)

chain = prompt | chat

chain.invoke({
    "purpose": "한국에서 친척이랑 할머니, 할아버지 산소에 갈 때"
})

AI: 
            한국에서 친척이랑 할머니, 할아버지 산소에 갈 때에는 다음과 같은 아이템을 추천합니다:
            -휴대용 의자 (산소에서 휴식을 취할 수 있는 휴대용 의자)
            -햇빛 차단용품 (모자, 선글라스, 선크림 등)
            -보조배터리 (전원이 필요한 경우를 대비하여)
            -휴대용 팬 (더운 날씨에 시원함을 유지하기 위해)
            -간식 (산소에서 즐길 수 있는 간식)

AIMessageChunk(content='AI: \n            한국에서 친척이랑 할머니, 할아버지 산소에 갈 때에는 다음과 같은 아이템을 추천합니다:\n            -휴대용 의자 (산소에서 휴식을 취할 수 있는 휴대용 의자)\n            -햇빛 차단용품 (모자, 선글라스, 선크림 등)\n            -보조배터리 (전원이 필요한 경우를 대비하여)\n            -휴대용 팬 (더운 날씨에 시원함을 유지하기 위해)\n            -간식 (산소에서 즐길 수 있는 간식)')